In [1]:
import pptk
from color_dict import *
import glob
import re
from PIL import Image 
import numpy as np
import progressbar

In [2]:
# This function will invoke a pptk viewer to render the points 
# Param: data: an array of points coordinate. Each point's coordinate should has the format of [x, y, z]
#       pointSize: the size of the point to be rendered on the screen
# Return: none

def displayPoints(data, color, pointSize):
    v = pptk.viewer(data)
    v.attributes(color)
    v.set(point_size=pointSize)

In [3]:
organData = [np.ones((1, 3)) for x in range(len(COLOR_DICT))]
organColor = [np.ones((1, 3)) for x in range(len(COLOR_DICT))]
unsureLoc = np.ones((1, 3))
pngPath = ".\masksToView_noisy\*.png"
path_list = [im_path for im_path in glob.glob(pngPath)]
path_list_parsed = [re.split('\\\\|\.', path) for path in path_list]
path_list_parsed_valid = [x for x in path_list_parsed if x[-1] == 'png']
path_list_parsed_valid = sorted(path_list_parsed_valid, key=lambda x:int(x[-2]))
volume = []
depthCounter = 0

for path in path_list_parsed_valid:
    s = "\\"
    path = [f for f in path if f != '']
    s = s.join(path)
    s = s[:-4] + '.png'
    maskArray = np.array(Image.open(s))
    
    for i in range(len(COLOR_DICT)):
        
        index = np.where(np.all(maskArray == COLOR_DICT[i], axis = 2))
        zCor = np.zeros(len(index[0])) + depthCounter
        
        if(np.array_equal(COLOR_DICT[i], Unsure_color)):
            # Make the unsure location blank so that it won't affect the whole volume 
            maskArray[index[0], index[1]] = Unlabelled_color
            curUnsure = np.dstack((zCor, index[0], index[1]))[0]
            unsureLoc = np.concatenate((unsureLoc, curUnsure))
        else:
            newCors = np.dstack((zCor, index[0], index[1]))[0]
            organData[i] = np.concatenate((organData[i], (newCors)))
        
    depthCounter += 1
    volume.append(maskArray)

In [4]:
organData[4].shape

(29495, 3)

In [5]:
volume = np.array(volume)
half_ws = 4

# Making a coordinate cube 
cube = np.mgrid[-half_ws:half_ws+1,-half_ws:half_ws+1,-half_ws:half_ws+1]
cube = np.stack((cube[0], cube[1], cube[2]), axis=3)
cube = np.reshape(cube, ((2*half_ws + 1)**3, 3))

with progressbar.ProgressBar(max_value=len(unsureLoc)) as bar:
    counter = 0
    
    # Iterate through every possible unsure points. Can be improved furthur
    for loc in unsureLoc[::500]:
        neighbors = cube + loc
        volumeBoundary = [volume.shape[0]-1, volume.shape[1]-1, volume.shape[2]-1]
        neighbors = np.clip(neighbors, [0, 0, 0], volumeBoundary)
        # Don't use int8
        neighbors = neighbors.astype('int16')
        neighborValue = volume[neighbors[:, 0], neighbors[:, 1], neighbors[:, 2]]
        neighborCount = np.zeros((len(COLOR_DICT)))

        for i in range(len(COLOR_DICT)):
            
            if(not np.array_equal(COLOR_DICT[i], Unsure_color)):
                index = np.where(np.all(neighborValue == COLOR_DICT[i], axis = 1))
                neighborCount[i] += len(index[0])
                
                # Uncomment the next line to show neighbor boxs 
                organData[3] = np.concatenate((organData[3], neighbors))
                #organData[3] = np.concatenate((organData[3], loc.reshape(1, 3)))
        
        # Ignore the unsure points who don't have any valid neighbors at all
        if(neighborCount.sum() > 0):
            correction = np.argmax(neighborCount)
            organData[correction] = np.append(organData[correction], loc.reshape(1, 3), axis=0)

        counter += 1
        bar.update(counter)

100% (103270 of 103270) |################| Elapsed Time: 0:00:03 Time:  0:00:03


In [6]:
organData[4].shape

(29511, 3)

In [7]:
for i in range(len(COLOR_DICT)):
    if(1 or not np.array_equal(COLOR_DICT[i], Unsure_color)):
        organColor[i] = np.zeros(organData[i].shape) + COLOR_DICT[i]

In [8]:
allOrganPts = np.concatenate(organData)
allOrganCol = np.concatenate(organColor)

In [9]:
displayPoints(allOrganPts[1::], allOrganCol[1::], 0.5)